In [1]:
import numpy as np
import matplotlib as plt

# 제 1고지 미분 자동 계산

###### 1단계 상자로서의 변수
###### 2단계 변수를 낳는 함수
###### 3단계 함수 연결
###### 4단계 수치 미분
###### 5단계 역전파 이론
###### 6단계 수동 역전파 이론
###### 7단계 역전파 자동화
###### 8단계 재귀에서 반복문으로
###### 9단계 함수를 더 편리하게
###### 10단계 테스트

## step 1 상자로서의 변수

In [2]:
# Variable 클래스 구현

class Variable :
    def __init__(self, data) :
        self.data = data

In [3]:


data = np.array(1.0)
x = Variable(data)

In [4]:
print(x.data)

1.0


In [5]:
x.data = np.array(2.0)

In [6]:
print(x.data)

2.0


## step2 변수를 낳는 함수

In [8]:
# Function 클래스 구현
class Function :
    def __call__(self, input) :
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [9]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [12]:
print(y)

In [13]:
print(y.data)

100


In [3]:
# real implementation

class Function :
    
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
        
    def forward(self, x) :
        raise NotImplementedError()

In [4]:
# square implementation

class Square(Function) :
    def forward(self, x) :
        return x ** 2

In [18]:
x = Variable(np.array(100))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
10000


## step 3 함수 연결

In [16]:
## exp 함수 구현

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)
    

In [20]:
A = Square()
B = Exp()
C = Square()

In [24]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step4 수치 미분

In [12]:
# 미분 개념
# 중앙 차분

def numerical_diff(f, x, eps = 1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [13]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [14]:
# 합성함수의 미분

def f(x) :
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [17]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [20]:
# 안 씀
# 근데 오차 커질 수 있다. 자릿수 누락과 함께
# 계산량이 너무 많다.



## step5 역전파 이론

In [21]:
# 손실함수인 y에 대한 각 파라미터의 그래디언트를 구하기 위해서임
# 순전파 & 역전파 
# 먼저 순전파하고 각 함수가 입력변수의 값 기억
# 그 값을 토대로 역전파 수행

# 통상값 (인퍼런스한 값) vs 미분값

## step6 수동 역전파

In [12]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None

In [13]:
# 미분을 계산하는 역전파
# forward 메소드 호출 시 건네받은 Variable 인스턴스 유지

class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()

In [6]:
## Square 
class Square(Function) :
    def forward(self, x) :
        y = x ** 2 
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [7]:
## Exp
class Exp(Function) :
    def forward(self, x) :
        y = np.exp(x)
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
        

In [27]:
# forward

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [28]:
# backward
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


## step7 역전파 자동화

In [55]:
# 변수와 함수의 관계 
# 변수에게 있어 함수는 창조자 혹은 부모입니다.
# 연결을 동적으로 하기
# Define-by-Run
# 그냥 링크드 리스트로 계산 그래프 표현

In [51]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

In [23]:
class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()    

In [49]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [58]:
# 1. 함수를 가져온다.
# 2. 함수의 입력을 가져온다.
# 3. 함수의 backward 메서드를 호출한다.

In [56]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [57]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [59]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [60]:
## backward 메서드 추가하기

In [7]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        f = self.creator
        if f is not None :
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [13]:
a = [1,2]
b = [1,2]
a is b

False

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## step8 재귀에서 반복문으로

In [22]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [26]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [27]:
print(y.creator)
print(b.creator)
print(a.creator)

## step9 함수를 더 편리하게

In [ ]:
# 사용자 편의를 위한 3가지 개선

#### 첫번째 개선 - 일일히 함수 크래스 정의 안하기

In [31]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

## -> 너무 귀찮아 ㅠㅠ

In [32]:
def square(x) :
    f = Square()
    return f(x)

def exp(x) :
    f = Exp()
    return f(x)

## -> 이것도 좀 귀찮아 ㅠㅠ    

In [8]:
def square(x) :
    return Square()(x)

def exp(x) :
    return Exp()(x)

In [34]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [35]:
# 연속으로 적용해보기
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


#### 두번째 개선 - backward 메소드 간소화 일일히 맨처음 기울기 1 제외

In [36]:
class Variable :
    def __init__(self, data) :
        self.data = data 
        self.grad = None
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [37]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


#### 세번째 개선 - ndarray만 취급하기

In [8]:
class Variable :
    def __init__(self, data) :
        if data is not None :
            if not isinstance(data, np.ndarray) :
                raise TypeError("{}은(는) 지원하지 않습니다.".format(type(data)))
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func) :
        self.creator = func

    def backward(self) :
        if self.grad is None :
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None :
                funcs.append(x.creator)

In [41]:
x = Variable(np.array(1.0))
x = Variable(None)

x = Variable(1)

TypeError: <class 'int'>은(는) 지원하지 않습니다.

In [42]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [43]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [9]:
def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x

In [45]:
import numpy as np
np.isscalar(np.float64(1.0))

True

In [46]:
np.isscalar(2.0)

True

In [47]:
np.isscalar(np.array(1.0))

False

In [51]:
np.isscalar(np.array([1,2,3]))

False

In [10]:
class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()    

## step10 테스트

In [13]:
def numerical_diff(f, x, eps=1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [14]:
import unittest

class SquareTest(unittest.TestCase) :
    def test_forward(self) :
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self) :
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self) :
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
        

unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 1.531s

OK


In [ ]:
## np.allclose는 아래와 같다.
## |a - b| <= (atol + rtol * |b|)

## 칼럼: 자동 미분

In [ ]:
# 미분 계산 3가지
## 수치 미분
## 기호 미분
## 자동 미분 - forward 모드 & reverse 모드 

# 제 2고지 자연스러운 코드로

###### 11단계 가변 길이 인수(순전파 편)
###### 12단계 가변 길이 인수(개선 편)
###### 13단계 가변 길이 인수(역전파 편)
###### 14단계 같은 변수 반복 사용
###### 15단계 복잡한 계산 그래프(이론편)
###### 16단계 복잡한 계산 그래프(구현편)
###### 17단계 메모리 관리와 순환 참조
###### 18단계 메모리 절약 모드
###### 19단계 변수 사용성 개선
###### 20단계 연산자 오버로드(1)
###### 21단계 연산자 오버로드(2)
###### 22단계 연산자 오버로드(3)
###### 23단계 패키지로 정의
###### 24단계 복잡한 함수의 미분

## step11 가변 길이 인수(순전파 편)

In [16]:
class Function :
    def __call__(self, inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs) :
        raise NotImplementedError()

    def backward(self, gys) :
        raise NotImplementedError()

In [17]:
class Add(Function) :
    def forward(self, xs) :
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [18]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


## step12 가변 길이 인수(개선 편)

In [ ]:
# 사용하는 사람과 구현하는 사람의 위한 개선

In [32]:
class Function :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

In [33]:
def f(*x) :
    print(x)

In [34]:
f(1,3,3,44444)

(1, 3, 3, 44444)


In [35]:
class Add(Function) :
    def forward(self, xs) :
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [36]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [38]:
class Function :
    def __call__(self, *inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple) :
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs :
            output.set_creator(self)
        self.inputs = inputs 
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

In [39]:
class Add(Function) :
    def forward(self, x0, x1) :
        y = x0 + x1
        return y

In [40]:
def add(x0, x1) :
    return Add()(x0, x1)

In [41]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## step13 가변 길이 인수(역전파 편)